# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Wednesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [3]:

import requests
from bs4 import BeautifulSoup
import csv

def scrape_imdb_movie_reviews(movie_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
    response = requests.get(movie_url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    reviews = []
    review_count = 0


    while review_count < 1000:
        for review in soup.find_all('div', class_='review-container'):
            if review_count >= 1000:
                break
            review_text = review.find('div', class_='text show-more__control').get_text().strip()
            reviews.append(review_text)
            review_count += 1

        next_button = soup.find('div', class_='load-more-data')
        if next_button:
            next_page = next_button.get('data-key')
            next_url = f'{movie_url}/reviews/_ajax?ref_=undefined&paginationKey={next_page}'
            response = requests.get(next_url, headers=headers)
            soup = BeautifulSoup(response.content, 'html.parser')
        else:
            break

    return reviews

def save_to_csv(data, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Review'])
        for item in data:
            writer.writerow([item])

if __name__ == "__main__":
    # Example taken URL for movies released in 2023 or 2024
    movie_urls = [
        "https://www.imdb.com/title/tt15398776/reviews/?ref_=tt_ql_2",
    ]

    all_reviews = []
    for movie_url in movie_urls:
        reviews = scrape_imdb_movie_reviews(movie_url)
        all_reviews.extend(reviews)

    save_to_csv(all_reviews, 'hanuman1_imdb_movie_reviews.csv')




# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [4]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
from google.colab import files
# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Reading the CSV file
df = pd.read_csv("hanuman1_imdb_movie_reviews.csv")

# Defining stopwords and stemmer
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Function to cleaning text
def clean_text(text):
    # Removing for noise
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Removing for numbers
    text = ''.join([i for i in text if not i.isdigit()])
    # Removing for stopwords
    text = ' '.join([word for word in text.split() if word.lower() not in stop_words])
    # Lowercasing  the  all texts data
    text = text.lower()
    # Stemming
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    # Lemmatization
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    return text

# Cleaning the text data
df['Cleaned_Review'] = df['Review'].apply(clean_text)

# Saving and downloading the clean data to a new CSV file
df.to_csv("hanuman1_imdb_movie_reviews_cleaned.csv", index=False)

files.download("hanuman1_imdb_movie_reviews_cleaned.csv")



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [2]:

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
import nltk
nltk.download('words')
from nltk import pos_tag, ne_chunk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.parse import DependencyGraph

# Sample clean text
clean_text = """
Write a python program to conduct syntax and structure analysis of the clean text you just saved above. The syntax and structure analysis includes:

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.
"""

# Tokenizimg the text into sentences
sentences = sent_tokenize(clean_text)

# Tokenizing each sentence into words and taging the parts of speech
pos_tags = []
noun_count = 0
verb_count = 0
adj_count = 0
adv_count = 0
for sentence in sentences:
    words = word_tokenize(sentence)
    tagged_words = pos_tag(words)
    pos_tags.append(tagged_words)
    for word, pos in tagged_words:
        if pos.startswith('N'):  # Nouns in sentence
            noun_count += 1
        elif pos.startswith('V'):  # Verb in sentence
            verb_count += 1
        elif pos.startswith('J'):  # Adjective in sentence
            adj_count += 1
        elif pos.startswith('R'):  # Adverb in sentence
            adv_count += 1

# POS tagging results
print(f"Total Nouns: {noun_count}")
print(f"Total Verbs: {verb_count}")
print(f"Total Adjectives: {adj_count}")
print(f"Total Adverbs: {adv_count}")
entities = []
for sentence in sentences:
    words = word_tokenize(sentence)
    tagged_words = pos_tag(words)
    named_entities = ne_chunk(tagged_words)
    for entity in named_entities:
        if isinstance(entity, nltk.Tree):
            entities.append(' '.join([word for word, pos in entity.leaves()]))

#named entities and their counts
print("\nNamed Entities:")
for entity, count in nltk.FreqDist(entities).items():
    print(f"{entity}: {count}")

sentences = sent_tokenize(clean_text)
for sentence in sentences:
    words = word_tokenize(sentence)
    tagged_words = pos_tag(words)
    print("\nConstituency Parsing Tree:")
    print(nltk.chunk.tree2conlltags(ne_chunk(tagged_words)))

    print("\nDependency Parsing Tree:")
    #print(nltk.parse.DependencyGraph.from_iterable(tagged_words).tree())




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


Total Nouns: 56
Total Verbs: 14
Total Adjectives: 5
Total Adverbs: 3

Named Entities:
Speech: 2
POS: 1
Tag Parts: 1
N: 1
V: 1
Adj: 1
Adv: 1
Dependency: 1
Entity Recognition: 1

Constituency Parsing Tree:
[('Write', 'VB', 'O'), ('a', 'DT', 'O'), ('python', 'NN', 'O'), ('program', 'NN', 'O'), ('to', 'TO', 'O'), ('conduct', 'VB', 'O'), ('syntax', 'NN', 'O'), ('and', 'CC', 'O'), ('structure', 'NN', 'O'), ('analysis', 'NN', 'O'), ('of', 'IN', 'O'), ('the', 'DT', 'O'), ('clean', 'JJ', 'O'), ('text', 'NN', 'O'), ('you', 'PRP', 'O'), ('just', 'RB', 'O'), ('saved', 'VBN', 'O'), ('above', 'RB', 'O'), ('.', '.', 'O')]

Dependency Parsing Tree:

Constituency Parsing Tree:
[('The', 'DT', 'O'), ('syntax', 'NN', 'O'), ('and', 'CC', 'O'), ('structure', 'NN', 'O'), ('analysis', 'NN', 'O'), ('includes', 'VBZ', 'O'), (':', ':', 'O'), ('(', '(', 'O'), ('1', 'CD', 'O'), (')', ')', 'O'), ('Parts', 'NNS', 'O'), ('of', 'IN', 'O'), ('Speech', 'NNP', 'B-GPE'), ('(', '(', 'O'), ('POS', 'NNP', 'B-ORGANIZATION'), 

# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [ ]:
# Write your response below
# In the above Assignment i wolud encountered to execute the code but with in built libraries it make easy to solve. Even it is type of learning to know
#required libraries to perform nlp functions and perform operations given. I have learn to extract and save the data set of customers given reviews for perticular movie
# and able to clean data set. I able to comple the assignment on time.
#Thank you.